In [2]:
from llama_index.core.workflow import (
    StartEvent,
    StopEvent,
    Workflow,
    step,
    Event,
    Context,
)
import asyncio
from llama_index.llms.openai import OpenAI
from llama_index.utils.workflow import draw_all_possible_flows

In [3]:
class FirstEvent(Event):
    first_output: str


class SecondEvent(Event):
    second_output: str
    response: str

In [18]:
class MyWorkflow(Workflow):
    @step
    async def step_one(self, ctx: Context, ev: StartEvent) -> FirstEvent:
        ctx.write_event_to_stream(Event(msg="Step one is happening", length=0))
        return FirstEvent(first_output="First step complete.")

    @step
    async def step_two(self, ctx: Context, ev: FirstEvent) -> SecondEvent:
        llm = OpenAI(model="gpt-4o-mini")
        generator = await llm.astream_complete(
            "Write 3 sentences about Vietnam"
        )
        async for response in generator:
            # Allow the workflow to stream this piece of response
            ctx.write_event_to_stream(Event(msg=response.delta))
        return SecondEvent(
            second_output="Second step complete, full response attached",
            response=str(response),
        )

    @step
    async def step_three(self, ctx: Context, ev: SecondEvent) -> StopEvent:
        ctx.write_event_to_stream(Event(msg="Step three is happening", length=0))
        return StopEvent(result="Workflow complete.")

In [19]:

async def main():
    w = MyWorkflow(timeout=30, verbose=False)
    task = asyncio.create_task(w.run(first_input="Start the workflow."))

    async for ev in w.stream_events():
        print(ev.msg, '----')

    final_result = await task
    print("Final result", final_result)

    # draw_all_possible_flows(MyWorkflow, filename="streaming_workflow.html")

from time import time

start = time()
await main()
print("Time taken", time() - start)

Step one is happening ----
 ----
Vietnam ----
 is ----
 a ----
 Southeast ----
 Asian ----
 country ----
 known ----
 for ----
 its ----
 rich ----
 history ----
, ----
 vibrant ----
 culture ----
, ----
 and ----
 stunning ----
 landscapes ----
, ----
 ranging ----
 from ----
 lush ----
 rice ----
 terraces ----
 to ----
 bustling ----
 cities ----
. ----
 The ----
 capital ----
, ----
 Hanoi ----
, ----
 is ----
 famous ----
 for ----
 its ----
 centuries ----
-old ----
 architecture ----
 and ----
 a ----
 rich ----
 culinary ----
 scene ----
, ----
 while ----
 Ho ----
 Chi ----
 Minh ----
 City ----
, ----
 formerly ----
 Sa ----
igon ----
, ----
 is ----
 a ----
 bustling ----
 metropolis ----
 that ----
 showcases ----
 the ----
 country's ----
 rapid ----
 modernization ----
. ----
 Vietnam ----
's ----
 diverse ----
 ethnic ----
 groups ----
 and ----
 traditions ----
 contribute ----
 to ----
 its ----
 unique ----
 cultural ----
 tapestry ----
, ----
 making ----
 it ----
 a

In [ ]:
async def main():
    w = MyWorkflow(timeout=30, verbose=False)
    task = asyncio.create_task(w.run(first_input="Start the workflow."))

    final_result = await task
    print("Final result", final_result)

    # draw_all_possible_flows(MyWorkflow, filename="streaming_workflow.html")


await main()